In [ ]:
#  Association Analysis - Apriori Analysis

In [3]:
!pip install mlxtend

In [1]:
import pandas as pd
from scipy.spatial.distance import cosine
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
# txn_data = pd.read_csv('C:\\Users\\jp\\Desktop\\testData\\item_txn_data.csv')
txn_data = pd.read_excel(r'C:\Users\Bhanu chennam\Desktop\cs\deepalogrithms\sample_txn.xlsx')

In [3]:
txn_data.head(40)

,txn,itm
0,1,Bread
1,1,Milk
2,2,Bread
3,2,Diaper
4,2,Beer
5,2,Eggs
6,3,Milk
7,3,Diaper
8,3,Beer
9,3,Coke


In [4]:
# grouping data frame items into list
list1 = txn_data.groupby('txn')['itm'].apply(list)
list1

txn
1                  [Bread, Milk]
2    [Bread, Diaper, Beer, Eggs]
3     [Milk, Diaper, Beer, Coke]
4    [Bread, Milk, Diaper, Beer]
5    [Bread, Milk, Diaper, Coke]
6           [Milk, Diaper, Beer]
Name: itm, dtype: object

In [5]:
# Convert into one-hot coded txn using mlxtend package
from mlxtend.preprocessing import TransactionEncoder
oht = TransactionEncoder()

In [6]:
oht_ary = oht.fit(list1).transform(list1)
txn_data_items = pd.DataFrame(oht_ary, columns=oht.columns_)
txn_data_items

,Beer,Bread,Coke,Diaper,Eggs,Milk
0,False,True,False,False,False,True
1,True,True,False,True,True,False
2,True,False,True,True,False,True
3,True,True,False,True,False,True
4,False,True,True,True,False,True
5,True,False,False,True,False,True


### Finding out Top Selling Products using Apriori algorithms


In [7]:
# Frequent ItemSets
frequent_itemsets = apriori(txn_data_items, min_support=0.5, use_colnames=True)

In [16]:
frequent_itemsets.sort_values('support', ascending=False)

,support,itemsets
2,0.833333,(Diaper)
3,0.833333,(Milk)
0,0.666667,(Beer)
1,0.666667,(Bread)
4,0.666667,"(Diaper, Beer)"
8,0.666667,"(Milk, Diaper)"
5,0.500000,"(Milk, Beer)"
6,0.500000,"(Diaper, Bread)"
7,0.500000,"(Milk, Bread)"
9,0.500000,"(Milk, Diaper, Beer)"


- Once identified the top selling products, next step is to look for all **Cross-Selling** opportunites by identifying which products are most frequently purchased with these top selling items.
- Cross Sell Possiblilities Calculation
- Cross Sell Popularity Calculation

In [17]:
# Apriori Association Rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Diaper),(Beer),0.833333,0.666667,0.666667,0.80,1.200,0.111111,1.666667
1,(Beer),(Diaper),0.666667,0.833333,0.666667,1.00,1.200,0.111111,inf
2,"(Milk, Diaper)",(Beer),0.666667,0.666667,0.500000,0.75,1.125,0.055556,1.333333
3,"(Milk, Beer)",(Diaper),0.500000,0.833333,0.500000,1.00,1.200,0.083333,inf
4,(Diaper),"(Milk, Beer)",0.833333,0.500000,0.500000,0.60,1.200,0.083333,1.250000
5,(Beer),"(Milk, Diaper)",0.666667,0.666667,0.500000,0.75,1.125,0.055556,1.333333


In [18]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=.7)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Diaper),(Beer),0.833333,0.666667,0.666667,0.80,1.200,0.111111,1.666667
1,(Beer),(Diaper),0.666667,0.833333,0.666667,1.00,1.200,0.111111,inf
2,(Beer),(Milk),0.666667,0.833333,0.500000,0.75,0.900,-0.055556,0.666667
3,(Bread),(Diaper),0.666667,0.833333,0.500000,0.75,0.900,-0.055556,0.666667
4,(Bread),(Milk),0.666667,0.833333,0.500000,0.75,0.900,-0.055556,0.666667
5,(Milk),(Diaper),0.833333,0.833333,0.666667,0.80,0.960,-0.027778,0.833333
6,(Diaper),(Milk),0.833333,0.833333,0.666667,0.80,0.960,-0.027778,0.833333
7,"(Milk, Diaper)",(Beer),0.666667,0.666667,0.500000,0.75,1.125,0.055556,1.333333
8,"(Milk, Beer)",(Diaper),0.500000,0.833333,0.500000,1.00,1.200,0.083333,inf
9,"(Diaper, Beer)",(Milk),0.666667,0.833333,0.500000,0.75,0.900,-0.055556,0.666667


In [19]:
# Filterd Rules
rules[(rules.support >=.6) & (rules.confidence >= .6) & (rules.lift >1) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Diaper),(Beer),0.833333,0.666667,0.666667,0.8,1.2,0.111111,1.666667
1,(Beer),(Diaper),0.666667,0.833333,0.666667,1.0,1.2,0.111111,inf
